# quake 3 server connector



## discord api


Authorize URL:

https://discord.com/oauth2/authorize?response_type=code&client_id=723583889779589221&scope=bot&permissions=2248399936&redirect_uri=https://discord.com/channels/@me



### channel message poller



#### the code

poll discord channel?

discord api?



In [4]:
var fs = require('fs')
var path = require('path')
var {request} = require('gaxios')
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
var credentials
var tokenPath
if(fs.existsSync('./discord-bot.txt')) {
    tokenPath = path.resolve('./discord-bot.txt')
} else {
    tokenPath = path.join(PROFILE_PATH, '.credentials/discord-bot.txt')
}
var token = fs.readFileSync(tokenPath).toString('utf-8').trim()

var DEFAULT_GUILD = process.env.DEFAULT_GUILD || '393252386426191873'
var DEFAULT_CHANNEL = process.env.DEFAULT_CHANNEL || '393252386426191875'
var DEFAULT_APPLICATION = process.env.DEFAULT_APPLICATION || '723583889779589221'
var DEFAULT_API = process.env.DEFAULT_API || 'https://discord.com/api/v8/'
var MESSAGE_TIME = process.env.DEFAULT_TIME || 1000 * 60 * 60 // 1 hour to respond
var DEFAULT_RATE = 2000
var previousRequest = 0

async function delay() {
    var now = (new Date()).getTime()
    previousRequest = now
    if(now - previousRequest < DEFAULT_RATE)
        await new Promise(resolve => setTimeout(resolve, DEFAULT_RATE - (now - previousRequest)))
    previousRequest = (new Date()).getTime()
}

async function authorizeUrl() {
    await delay()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}gateway/bot`
    })
    return result.data
}

async function userGuilds(userId = '@me') {
    await delay()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}users/${userId}/guilds`
    })
    return result.data
}

async function getGuildRoles(guildId = DEFAULT_GUILD) {
    await delay()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}guilds/${guildId}/roles`
    })
    return result.data
}

async function userChannels(userId = '@me') {
    await delay()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}channels/${userId}`
    })
    return result.data
}

async function userConnections(userId = '@me') {
    await delay()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}users/${userId}/connections`
    })
    return result.data
}

async function guildChannels(guildId = DEFAULT_GUILD) {
    await delay()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}guilds/${guildId}/channels`
    })
    return result.data
}

async function channelMessages(channelId = DEFAULT_CHANNEL) {
    await delay()
    var params = {
        limit: 100,
        after: (BigInt(Date.now() - 1420070400000 - MESSAGE_TIME) << BigInt(22)).toString()
    };
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}channels/${channelId}/messages`,
        params
    })
    return result.data
}

async function triggerTyping(channelId = DEFAULT_CHANNEL) {
    await delay()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'POST',
        url: `${DEFAULT_API}channels/${channelId}/typing`
    })
    return result.data
}

async function createMessage(message, channelId = DEFAULT_CHANNEL) {
    await delay()
    var params = typeof message == 'string' ? ({
        'content': message
    }) : message
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`,
            'Content-Type': 'application/json'
        },
        method: 'POST',
        url: `${DEFAULT_API}channels/${channelId}/messages`,
        data: JSON.stringify(params)
    })
    return result.data
}

async function updateMessage(message, messageId, channelId = DEFAULT_CHANNEL) {
    await delay()
    var params = typeof message == 'string' ? ({
        'content': message
    }) : message
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`,
            'Content-Type': 'application/json'
        },
        method: 'PATCH',
        url: `${DEFAULT_API}channels/${channelId}/messages/${messageId}`,
        data: JSON.stringify(params)
    })
    return result.data
}

async function registerCommand(cmd, desc, guildId = null) {
    await delay()
    // TODO: guild specific commands
    //url = "https://discord.com/api/v8/applications/<my_application_id>/guilds/<guild_id>/commands"
    var json
    if(typeof cmd == 'object') {
        json = cmd
    } else {
        json = {
            'name': cmd,
            'description': desc,
            'options': []
        }
    }
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`,
            'Content-Type': 'application/json'
        },
        method: 'POST',
        url: `${DEFAULT_API}applications/${DEFAULT_APPLICATION}/commands`,
        data: JSON.stringify(json)
    })
    return result.data
}

async function interactionResponse(interactionId, interactionToken) {
    await delay()
    var json = {
        'type': 5
    }
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`,
            'Content-Type': 'application/json'
        },
        method: 'POST',
        url: `${DEFAULT_API}interactions/${interactionId}/${interactionToken}/callback`,
        data: JSON.stringify(json)
    })
    return result.data
}

async function getCommands(guildId = null) {
    await delay()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}applications/${DEFAULT_APPLICATION}/commands`
    })
    return result.data
}

async function updateInteraction(message, interactionId, interactionToken) {
    await delay()
    var json = typeof message == 'string' ? ({
            'content': message
        }) : message
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`,
            'Content-Type': 'application/json'
        },
        method: 'PATCH',
        url: `${DEFAULT_API}webhooks/${DEFAULT_APPLICATION}/${interactionToken}/messages/@original`,
        data: JSON.stringify(json)
    })
    return result.data
}

async function createThread(name, channelId = DEFAULT_CHANNEL) {
    await delay()
    var json = {
        'name': name,
        'type': 11,
        'auto_archive_duration': 60
    }
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`,
            'Content-Type': 'application/json'
        },
        method: 'POST',
        url: `${DEFAULT_API}channels/${channelId}/threads`,
        data: JSON.stringify(json)
    })
    return result.data
}

async function archivedThreads(channelId = DEFAULT_CHANNEL) {
    await delay()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}channels/${channelId}/threads/archived/public`
    })
    return result.data
}

async function activeThreads(channelId = DEFAULT_CHANNEL) {
    await delay()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}channels/${channelId}/threads/active`
    })
    return result.data
}

async function deleteCommand(commandId) {
    await delay()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'DELETE',
        url: `${DEFAULT_API}applications/${DEFAULT_APPLICATION}/commands/${commandId}`
    })
    return result.data
}

async function getPins(channelId = DEFAULT_CHANNEL) {
    await delay()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}channels/${channelId}/pins`
    })
    return result.data
}

async function pinMessage(messageId, channelId = DEFAULT_CHANNEL) {
    await delay()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'PUT',
        url: `${DEFAULT_API}channels/${channelId}/pins/${messageId}`
    })
    return result.data
}

async function unpinMessage(messageId, channelId = DEFAULT_CHANNEL) {
    await delay()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'DELETE',
        url: `${DEFAULT_API}channels/${channelId}/pins/${messageId}`
    })
    return result.data
}



module.exports = {
    authorizeUrl,
    userGuilds,
    userChannels,
    userConnections,
    guildChannels,
    getGuildRoles,
    channelMessages,
    triggerTyping,
    createMessage,
    updateMessage,
    registerCommand,
    getCommands,
    interactionResponse,
    updateInteraction,
    createThread,
    archivedThreads,
    activeThreads,
    deleteCommand,
    getPins,
    pinMessage,
    unpinMessage
}


{
  userGuilds: [AsyncFunction: userGuilds],
  guildChannels: [AsyncFunction: guildChannels],
  channelMessages: [AsyncFunction: channelMessages],
  triggerTyping: [AsyncFunction: triggerTyping],
  createMessage: [AsyncFunction: createMessage]
}

#### test sending a discord message?



In [ ]:
var discordApi = importer.import('discord api')
var {authorizeGateway} = importer.import('authorize discord')

async function testMessage()
{
    var discordSocket = await authorizeGateway()
    await discordApi.createMessage('beep boop', '752568660819837019')
    discordSocket.close()
}

module.exports = testMessage


### authorize discord oauth



#### the code

authorize discord?


In [ ]:
var fs = require('fs')
var path = require('path')
var WebSocket = require('ws')
var {request} = require('gaxios')
var importer = require('../Core')
var {authorizeUrl, interactionResponse} = importer.import('discord api')
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
var tokenPath
if(fs.existsSync('./discord-bot.txt')) {
    tokenPath = path.resolve('./discord-bot.txt')
} else {
    tokenPath = path.join(PROFILE_PATH, '.credentials/discord-bot.txt')
}
var token = fs.readFileSync(tokenPath).toString('utf-8').trim()

var DEFAULT_API = process.env.DEFAULT_API || 'https://discord.com/api/v6/'

var heartbeat
var ws = false
var wsConnecting = false
var cancelConnection
var seq = 0
var privateChannels = {}
var interactions = {}
var shouldReconnect = false

function sendHeartbeat() {
    if(!ws) return
    console.log('Sending heartbeat')
    ws.send(JSON.stringify({
        op: 1,
        d: seq
    }))
    cancelConnection = setTimeout(() => ws ? ws.close() : false, 4000)
}

async function authorizeGateway() {
    var result
    if(wsConnecting) {
        await new Promise(resolve => {
            var authorizeWait
            var authorizeCount = 0
            authorizeWait = setInterval(() => {
                if(typeof ws == 'object' &&
                    ws.readyState == 1
                    || authorizeCount == 30) {
                    clearInterval(authorizeWait)
                    resolve()
                } else {
                    authorizeCount++
                }
            }, 100)
        })
    }
    if(typeof ws == 'object' && ws.readyState == 1)
        return // already connected, no need to continue
    wsConnecting = true
    try {
        result = await authorizeUrl()
    } catch (e) {
        console.log(e.message)
        ws = false
        return
    }
    ws = new WebSocket(result.url)
    await new Promise(resolve => {
        ws.on('open', () => {
            wsConnecting = false
            shouldReconnect = false
            console.log('Connecting to Discord')
            resolve()
        })
    })
    var identified = false
    ws.on('message', (message) => {
        var msgBuff = new Buffer.from(message)
        var gateway = JSON.parse(msgBuff.toString('utf-8'))
        if(gateway.s) seq = gateway.s
        if(gateway.d && gateway.d.seq) seq = gateway.d.seq
        if(gateway.op == 10) {
            heartbeat = setInterval(sendHeartbeat, gateway.d.heartbeat_interval)
            ws.send(JSON.stringify({
                op: 2,
                intents: ['DIRECT_MESSAGES', 'GUILD_MESSAGES', 'GUILDS'],
                d: {
                    token: token,
                    properties: {
                        "$os": "linux",
                        "$browser": "jupyter",
                        "$device": "quake3"
                    }
                }
            }))
            return
        } else if (gateway.op === 7) {
            shouldReconnect = true
            return
        } else if (gateway.op === 0 || gateway.op === 9) {
            identified = true
            if(gateway.t == 'MESSAGE_CREATE' 
                // guild ID can only be null if it is a personal message
                && typeof gateway.d.guild_id == 'undefined') {
                console.log(gateway)
                privateChannels[gateway.d.channel_id] = Date.now()
            }
            if(gateway.t == 'INTERACTION_CREATE') {
                if(typeof interactions[gateway.d.channel_id] == 'undefined')
                    interactions[gateway.d.channel_id] = []
                interactions[gateway.d.channel_id].push(gateway.d)
                interactionResponse(gateway.d.id, gateway.d.token)
            }
            return
        } else if (gateway.op === 11) {
            clearTimeout(cancelConnection)
            return
        }
        console.log(gateway)
    })
    var timer
    ws.on('close', () => {
        console.log('Discord disconnected')
        if(timer) clearInterval(timer)
        clearInterval(heartbeat)
        ws.close()
        ws = false
        if(shouldReconnect)
            setTimeout(authorizeGateway, 1000)
        return
    })
    await new Promise(resolve => {
        timer = setInterval(() => {
            if(identified) {
                clearInterval(timer)
                resolve()
            }
        }, 1000)
    });
    return ws
}

module.exports = {
    authorizeGateway,
    privateChannels,
    interactions
}


### synchronize discord commands

sync discord commands?


#### the code


In [ ]:
var importer = require('../Core')
var {authorizeGateway} = importer.import('authorize discord')
var {registerCommand, getCommands, deleteCommand} = importer.import('discord api')

async function syncCommands() {
    await authorizeGateway()
    var commandResult = (await getCommands())
    var commands = commandResult.map(command => command.name)
    if(commands.includes('hello-orbb'))
    await deleteCommand(commandResult.filter(c => c.name == 'hello-orbb')[0].id)
    if(!commands.includes('hello'))
    await registerCommand('hello', 'Check if Orbb is awake.')
    if(!commands.includes('challenge'))
    await registerCommand({
        'name': 'challenge',
        'description': 'Challenges another user to match, Orbb waits for the thumbs up.',
        'options': [
            {
                'name': 'opponent-id',
                'description': 'Name of the player you want to challenge for 1v1.',
                'required': true,
                'type': 6
            },
            {
                'name': 'map',
                'description': 'Name of the map to start on the server.',
                'required': true,
                'type': 3
            }
        ]
    })
    if(!commands.includes('connect'))
    await registerCommand({
        'name': 'connect',
        'description': 'RCon Connect to a Quake 3 server for remote administration over Discord.',
        'options': [
            {
                'name': 'server-address',
                'description': 'The IP address or domain name of the server to connect to including port.',
                'required': true,
                'type': 3
            }
        ]
    })
    if(!commands.includes('rcon'))
    await registerCommand({
        'name': 'rcon',
        'description': 'Set the password for future RCon commands, or send an rcon command to the connected server.',
        'options': [
            {
                'name': 'rcon-password',
                'description': 'Password to use with future RCon commands.',
                'required': true,
                'type': 3
            },
            {
                'name': 'rcon-command',
                'description': 'Send the following RCon command to the server.',
                'required': false,
                'type': 3
            }
        ]
    })
    if(!commands.includes('config'))
    await registerCommand({
        'name': 'config',
        'description': 'Execute a config file on the remote Quake 3 server after using /connect command.',
        'options': [
            {
                'name': 'config-name',
                'description': 'Name of the config script to execute',
                'required': true,
                'type': 3
            }
            // TODO: not required and list availabe config scripts through engine
        ]
    })
    if(!commands.includes('map'))
    await registerCommand({
        'name': 'map',
        'description': 'Starts a server with the specified map and sends you a personal message when the server is ready.',
        'options': [
            {
                'name': 'map-name',
                'description': 'Name of the map to run the server.',
                'required': true,
                'type': 3
            }
        ]
    })
    return await getCommands()
}


module.exports = syncCommands


### respond discord commands?

Interesting how this turned out. Commands use the :thumbsup: symbol as a confirmation. Other than that it is `command` followed by `response`. The code tag for `beep boop` contains a hidden language specifier that contains the original commands `message.id`. Because nonce isn't working, it uses a code block.



#### the code



In [ ]:
var importer = require('../Core')
var discordApi = importer.import('discord api')
var {authorizeGateway, privateChannels, interactions} = importer.import('authorize discord')
var {
    discordCommands,
    challengeCommand,
    configCommand,
    connectCommand,
    rconCommand,
    chatCommand,
} = importer.import('discord commands')

var DEFAULT_USERNAME = 'Orbb'

function interpretCommand(message) {
    return Object.keys(discordCommands)
       .filter(k => message.content.match(discordCommands[k])
              || (message.attachments && message.attachments.filter(a => a.filename.match(discordCommands[k])).length > 0)
              || (message.embeds && message.embeds.filter(e => (e.title && e.title.match(discordCommands[k]))
                                                          || (e.description && e.description.match(discordCommands[k]))).length > 0))
}

async function readAllCommands(specificChannel) {
    // matching format  @megamind  challenge freon dm17 , :thumbsup:   :thumbsdown: .
    var private = false
    var messages = []
    var responses = []
    var channels = []
    var commands = []
    var launches = []
    
    if(specificChannel == '@me') {
        // only read channel if it was updated within the last hour
        var userChannels = Object
            .keys(privateChannels)
            .filter(k => privateChannels[k] > Date.now() - 1000 * 60 * 60)
            .map(k => ({id: k}))
        channels.push.apply(channels, userChannels)
        specificChannel = ''
        private = true
    } else {
        var guilds = await discordApi.userGuilds()
        console.log(`Reading ${guilds.length} guilds`)
        for(var i = 0; i < guilds.length; i++) {
            channels.push.apply(channels, await discordApi.guildChannels(guilds[i].id))
        }
    }
    
    console.log(`Reading ${channels.length} channels`)
    for(var i = 0; i < channels.length; i++) {
        if(!specificChannel
           || channels[i].id == specificChannel
           || (typeof specificChannel == 'string'
              && (specificChannel.length === 0
                 || (channels[i].name
                     && channels[i].name.match(new RegExp(specificChannel, 'ig'))
                    )
                 )
              )
          ) {
            console.log(`Reading ${channels[i].name}`)
            messages.push.apply(messages, await discordApi.channelMessages(channels[i].id))
        }
    }
    
    // find commands in channel history
    console.log(`Reading ${messages.length} messages`)
    for(var j = 0; j < messages.length; j++) {
        var applicableCommands = interpretCommand(messages[j])
        if(applicableCommands.length > 0
          && messages[j].author.username != DEFAULT_USERNAME) {
            messages[j].commands = applicableCommands
            messages[j].private = private
            commands.push(messages[j])
            if((messages[j].reactions || [])
                .filter(a => a.emoji.name == '\u{1F44D}').length > 0) {
                launches.push(messages[j])
            }
        }
        if(messages[j].content.match(/```BOT/ig)) {
            responses.push(messages[j])
            if((messages[j].reactions || [])
                .filter(a => a.emoji.name == '\u{1F44D}').length > 0) {
                var l = messages.filter(m => messages[j].content.match('```BOT'+m.id))[0]
                if(!l) continue
                l.launching = true
                l.reactions = l.reactions || []
                l.reactions.push.apply(l.reactions, messages[j].reactions)
                if(l) launches.push(l)
            }
        }
    }
        
    // find all commands in interactions
    var interactionsCount = Object.keys(interactions)
        .reduce((sum, i) => {return sum + interactions[i].length}, 0)
    console.log(`Reading ${Object.keys(interactions).length} channels with ${interactionsCount} interactions`)
    Object.keys(interactions).forEach(i => {
        for(var c in interactions[i]) {
            interactions[i][c].commands = [interactions[i][c].data.name.toUpperCase()]
            interactions[i][c].author = interactions[i][c].member.user
            interactions[i][c].content = interactions[i][c].data.name + ' '
                + (interactions[i][c].data.options || []).map(o => o.value).join(' ')
            interactions[i][c].interaction = true
            commands.push(interactions[i][c])
        }
        interactions[i] = []
    })


    // exclude commands that already got a response
    return commands
        .filter(c => responses.filter(r => r.content.match(new RegExp('```BOT'+c.id))).length === 0)
        .concat(launches)
        .filter(c => responses.filter(r => r.content.match(new RegExp('```BOT'+c.id+'L'))).length === 0)
        .filter((c, i, arr) => arr.indexOf(c) === i)
}

async function respondCommand(specificChannel) {
    await authorizeGateway()
    var commands = await readAllCommands(specificChannel)
    for(var i = 0; i < commands.length; i++) {
        if(commands[i].commands.includes('CHALLENGE'))
            await challengeCommand(commands[i])
        else if(commands[i].commands.includes('CONFIG')) 
            await configCommand(commands[i])
        else if(commands[i].commands.includes('CONNECT'))
            await connectCommand(commands[i])
        else if(commands[i].commands.includes('RCON'))
            await rconCommand(commands[i])
        else if(commands[i].commands.includes('HELLO'))
            await chatCommand(commands[i])
        else if(commands[i].private) {
            console.log('Unknown command', commands[i])
            //await unknownCommand(commands[i])
        }
    }
}

module.exports = respondCommand


#### test specific channel?



In [ ]:
var importer = require('../Core')
var respondCommand = importer.import('respond discord commands')

module.exports = function testChannel(channel) {
    respondCommand(channel)
    
}


### challenge discord command?



#### the code



In [ ]:
var importer = require('../Core')
var discordApi = importer.import('discord api')
var serverApi = importer.import('quake 3 server commands')

var CHALLENGE = /(@[^:@\s]+\s*chall?[ae]nge|chall?[ae]nge\s*@[^:@\s]+)\s*([^:@\s]*?)\s*([^:@\s]*?)/ig
var DEFAULT_HOST = process.env.DEFAULT_HOST || 'http://quakeiiiarena.com/play/'
var MODS = typeof process.env.DEFAULT_MODS == 'string'
    ? JSON.parse(process.env.DEFAULT_MODS)
    : [
        'baseq3',
        'freon'
    ]

async function challengeCommand(command) {
    if(!command.private && (!command.mentions || command.mentions.length === 0))
        return
    var options = CHALLENGE.exec(command.content)
    var launch = (options ? options[2] : '') || ''
    var map = (options ? options[3] : '') || ''
    var message = 'I read you'
    var instruction = ''
    if(!MODS.includes(launch) && map.length === 0) {
        map = launch
        launch = ''
    }
    if(map.length === 0) {
        map = 'q3dm17'
    }
    if(launch.length == 0) {
        instruction += ', assuming baseq3 on map ' + map
    } else if(command.launching) {
        instruction += ' ' + launch + ' on map ' + map
    }
    if(!command.launching && !command.content.match(/:thumbsup:/ig)) {
        message = 'Waiting for reaction'
        instruction += ', react with :thumbsup: to launch'
    }
    if(command.launching) {
        message = 'Launching'
        await discordApi.createMessage(message + instruction + '\n```BOT'+command.id+'L\nbeep boop\n```\n', command.channel_id)
        await discordApi.triggerTyping(command.channel_id)
        var masters = await serverApi.listMasters(void 0, void 0, false)
        if(masters.length === 0) {
            await discordApi.createMessage(`Boo hoo, no servers available. :cry:` 
                + '\n```BOT'+command.id+'L\nbeep boop\n```\n', command.channel_id)
            return
        }
        await serverApi.sendRcon(masters[0].ip, masters[0].port, '\exec ' + launch + '.cfg')
        await serverApi.sendRcon(masters[0].ip, masters[0].port, '\map ' + map)
        await new Promise(resolve => setTimeout(resolve, 1000))
        await discordApi.createMessage(`Match is ready ${DEFAULT_HOST}?connect%20${masters[0].ip}:${masters[0].port} (${masters[0].ip}:${masters[0].port})`
                                       + '\n```BOT'+command.id+'L\nbeep boop\n```\n', command.channel_id)
    } else if (instruction.length > 0) {
        await discordApi.createMessage(message + instruction + '\n```BOT'+command.id+'\nbeep boop\n```\n', command.channel_id)
    }
}

module.exports = challengeCommand


### direct messages and discord rcon



#### the code

discord bot?

discord commands?


In [ ]:
var ip6addr = require('ip6addr')
var importer = require('../Core')
var challengeCommand = importer.import('challenge discord command')
var discordApi = importer.import('discord api')
var {
    getInfo, sendRcon, nextInfoResponse,
    nextPrintResponse
} = importer.import('quake 3 server commands')
var formatQuake3Response = importer.import('format quake 3 response')
var removeCtrlChars = importer.import('remove ctrl characters')

var personality = [
    'Yeehaw!',
    'Balls to wall!',
    'Do it to it!',
    'Got it!',
    'Let\'s play!',
    'Roger that!',
    'I read you!',
    'Buenos Dias!'
]

var lose = [
    'Error. Error.',
    'Oops.',
    'Boo hoo!',
    'Phooey!',
    'Au revoir, mon amis.',
    '#*&^@#!!',
]

var discordCommands = {
    CHALLENGE: /^[!\\\/]?(<@[^:@\s]+>\s*chall?[ae]nge|chall?[ae]nge\s*<@[^:@\s]+>)\s*([^:@\s]*?)\s*([^:@\s]*?)/ig,
    CONNECT: /^[!\\\/]?(rcon)?conn?ect\s*([0-9\.a-z-_]+(:[0-9]+)*)$/ig,
    RCON: /^[!\\\/]?rcon(pass?wo?rd)?\s+([^"\s]+)\s*(.*)$/ig,
    DISCONNECT: /[!\\\/]?disconn?ect/ig,
    CONFIG: /^[!\\\/]?(\w*)(\.cfg|config|configure)/ig,
    LOAD: /^[!\\\/]?(load|map)\s*(\w*)/ig,
    COMMAND: /^[!\\\/]/ig,
    HELLO: /^[!\\\/](\w\s*){0,2}hello(\w\s*){0,2}/ig,
    UNKNOWN: /.*/ig,
}

async function configCommand(command) {
    if(!command.attachments && !command.embed) return
    var user = command.author.username
    var options = discordCommands.CONFIG.exec(command.content)
    var options2 = command.attachments
        .map(a => discordCommands.CONFIG.exec(a.filename))
        .filter(a => a)[0]
    var name = options ? options[1] : options2 ? options2[1] : ''
        .replace(options[2], '')
        .replace(options2[2], '')
        .replace(new RegExp(user, 'ig'), '')
        .replace(/[^0-9-_a-z]/ig, '-')
    if(name.length === 0) {
        await discordApi.createMessage(`Couldn't compute filename.` + '\n```BOT'+command.id+'\nbeep boop\n```\n', command.channel_id)
        return
    }
    var file = 'player-' + user + '-' + name + '.cfg'
    await discordApi.triggerTyping(command.channel_id)
    // TODO: remote post
    //await remoteGet(command.attachments[0].url, file, '/home/freonjs/baseq3-cc/conf/')
    await discordApi.createMessage(`exec conf/player-${user}-${name}` + '\n```BOT'+command.id+'\nbeep boop\n```\n', command.channel_id)
}

var userLogins = {}
// username: {address, password, lastUsed, }
async function connectCommand(command) {
    // TODO: record last address and password given
    var user = command.author.username
    var options = discordCommands.CONNECT.exec(command.content)
    if(typeof userLogins[user] == 'undefined')
        userLogins[user] = {}
    userLogins[user] = {
        address: options[2] || userLogins[user].address || 'quakeIIIarena.com',
        password: userLogins[user].password || 'password123!'
    }
    // TODO: try to connect to server and respond with a getinfo print out
    await discordApi.triggerTyping(command.channel_id)
    var match = (/^(.*?):*([0-9]+)*$/ig).exec(userLogins[user].address)
    await getInfo(match[1], parseInt(match[2]) || 27960)
    var info = await nextInfoResponse()
    var filteredKeys = Object.keys(info)
        .filter(k => k != 'challenge'
                && k != 'hostname'
                && k != 'sv_hostname'
                && k != 'mapname'
                && k != 'clients'
                && k != 'g_humanplayers'
                && k != 'sv_maxclients'
                && k != 'ip'
                && k != 'port')
        .map(k => removeCtrlChars(k))
    var filteredValues = filteredKeys
        .map(k => removeCtrlChars(info[k]))
    var json = {
        content: '\n```BOT'+command.id+'\nbeep boop\n```\n',
        embeds: [{
            title: removeCtrlChars(info.sv_hostname || info.hostname || info.gamename || info.game || ''),
            description: info.ip + ':' + info.port,
            color: 0xdda60f,
            fields: [
                {
                    name: 'Map',
                    value: info.mapname,
                    inline: false
                },
                {
                    name: 'Players',
                    value: info.clients + ' (' + (info.g_humanplayers || '?') + ' humans)' + '/' + info.sv_maxclients,
                    inline: false
                },
                {
                    name: 'Key',
                    value: '```http\n' + filteredKeys.join('\n') + '```',
                    inline: true
                },
                {
                    name: 'Value',
                    value: '```yaml\n' + filteredValues.join('\n') + '```',
                    inline: true
                }
            ]
        }]
    }
    
    if(command.interaction)
        await discordApi.updateInteraction(json, command.id, command.token)    
    else
        await discordApi.createMessage(json, command.channel_id)    
}

async function rconCommand(command) {
    var user = command.author.username
    var options = discordCommands.RCON.exec(command.content)
    if(typeof userLogins[user] == 'undefined')
        userLogins[user] = {}
    userLogins[user] = {
        address: userLogins[user].address || 'quakeIIIarena.com',
        password: options[2] || userLogins[user].password || 'password123!'
    }
    await discordApi.triggerTyping(command.channel_id)
    var match = (/^(.*?):*([0-9]+)*$/ig).exec(userLogins[user].address)
    await sendRcon(match[1], parseInt(match[2]) || 27960,
             options[3] && options[3].length > 0
                 ? options[3]
                 : 'cmdlist',
             userLogins[user].password)
    var response = await nextPrintResponse()
    response = formatQuake3Response(response.content, command, response)
    if(typeof response == 'string')
        response += '\n```BOT'+command.id+'\nbeep boop\n```\n'
    else if(typeof response == 'object')
        response.content = '\n```BOT'+command.id+'\nbeep boop\n```\n'
    if(command.interaction)
        await discordApi.updateInteraction(response, command.id, command.token)    
    else
        await discordApi.createMessage(response, command.channel_id)    
}

async function chatCommand(command) {
    if(command.interaction)
        await discordApi.updateInteraction(`Hello.` + '\n```BOT'+command.id+'\nbeep boop\n```\n', command.id, command.token)
    else
        await discordApi.createMessage(`Hello.` + '\n```BOT'+command.id+'\nbeep boop\n```\n', command.channel_id)
    return
}

module.exports = {
    discordCommands,
    challengeCommand,
    configCommand,
    connectCommand,
    rconCommand,
    chatCommand,
}


## quake 3 commands



### quake 3 server commands

The formatting for these functions look a little odd. This is because of the nature of the master server list. In some cases, like the challenge command, which server is selected doesn't matter just as long as there is one available. But in the case of monitoring services, we want a specific match. This is what commands are expected to be run and then waited on either synchronously or asynchronously.



#### the code

quake 3 server commands?


In [ ]:
var path = require('path')
var fs = require('fs')
var zlib = require('zlib')
var dgram = require('dgram')
var udpClient = dgram.createSocket('udp4')
udpClient.on('message', updateInfo)

var importer = require('../Core')
var mdfour = importer.import('md4 checksum')
var {
    getServersResponse, statusResponse, infoResponse
} = importer.import('quake 3 server responses')
var lookupDNS = importer.import('dns lookup')
var {
    compressMessage,
    writeBits
} = importer.import('huffman decode')
var decodeClientMessage = importer.import('decode client message')


var MAX_TIMEOUT = process.env.DEFAULT_TIMEOUT || 10000
var MAX_RELIABLE_COMMANDS = 64
var DEFAULT_MASTER = process.env.DEFAULT_MASTER || '207.246.91.235' || '192.168.0.4'
var DEFAULT_PASS = process.env.DEFAULT_PASS || 'password123!'

var masters = []
var nextResponse = {
    nextInfo: null,
    nextStatus: null,
    nextServer: null,
    nextPrint: null,
}

function mergeMaster(master) {
    var found = false
    masters.forEach((ma, i) => {
        if(ma['ip'] == master['ip'] && ma['port'] == master['port']) {
            found = true
            Object.assign(masters[i], master)
            Object.assign(master, masters[i])
            return false
        }
    })
    if(!found)
        masters.push(master)
    return master
}

async function updateInfo(m, rinfo) {
    var master = mergeMaster({
        ip: rinfo.address,
        port: rinfo.port
    })
    if(m[0] == 255 && m[1] == 255 && m[2] == 255 && m[3] == 255)
        m = m.slice(4, m.length)
    else {
        if(master.connected) {
            master.channel = master.channel || {}
            var commandNumber = master.channel.commandSequence
            var channel = await decodeClientMessage(m, master.channel)
            if(channel === false) {
                return
            }
            //console.log(channel)
            if (channel.messageType == 2) { // svc_gamestate
                nextResponse.nextGamestate = 
                master.nextResponse.nextGamestate = channel
            } else if (channel.messageType == 7) { // svc_snapshot
                nextResponse.nextSnapshot = 
                master.nextResponse.nextSnapshot = channel
            } else if (channel.messageType > 0) {
            }
            if(commandNumber < channel.commandSequence) {
                for(var j = commandNumber + 1; j <= channel.commandSequence; j++) {
                    var index = j & (MAX_RELIABLE_COMMANDS-1)
                    if((channel.serverCommands[index] + '').match(/^chat /i)) {
                        nextResponse.nextChat = 
                        master.nextResponse.nextChat = channel.serverCommands[index] + ''
                    }
                    console.log('serverCommand:', j, channel.serverCommands[index])
                }
            }
            // always respond with input event
            // response to snapshots automatically and not waiting,
            //   so new messages can be received
            sendSequence(rinfo.address, rinfo.port, channel)
            nextResponse.nextChannel = 
            master.nextResponse.nextChannel = channel
            return
        }
    }
        
    if(m.slice(0, 'getserversResponse'.length).toString('utf-8').toLowerCase() == 'getserversresponse'
      || m.slice(0, 'getserversExtResponse'.length).toString('utf-8').toLowerCase() == 'getserversextresponse') {
        var masters = getServersResponse(m)
        for(var m in masters) {
            nextResponse.nextServer = 
            master.nextResponse.nextServer = mergeMaster(masters[m])
            await getStatus(masters[m].ip, masters[m].port)
        }
    } else if (m.slice(0, 'statusResponse'.length).toString('utf-8').toLowerCase() == 'statusresponse') {
        var status = mergeMaster(Object.assign(statusResponse(m), {
            ip: rinfo.address,
            port: rinfo.port
        }))
        nextResponse.nextStatus = 
        master.nextResponse.nextStatus = status
    } else if (m.slice(0, 'infoResponse'.length).toString('utf-8').toLowerCase() == 'inforesponse') {
        var info = mergeMaster(Object.assign(infoResponse(m), {
            ip: rinfo.address,
            port: rinfo.port
        }))
        nextResponse.nextInfo = 
        master.nextResponse.nextInfo = info
    } else if (m.slice(0, 'print'.length).toString('utf-8') == 'print') {
        var print = mergeMaster(Object.assign({
            content: m.slice('print'.length).toString('utf-8')
        }, {
            ip: rinfo.address,
            port: rinfo.port
        }))
        nextResponse.nextPrint = 
        master.nextResponse.nextPrint = print
    } else if (m.slice(0, 'challengeResponse'.length).toString('utf-8').toLowerCase() == 'challengeresponse') {
        var challenge = mergeMaster(Object.assign({
            challenge: m.slice('challengeResponse'.length).toString('utf-8').trim().split(/\s+/ig)[0]
        }, {
            ip: rinfo.address,
            port: rinfo.port
        }))
        nextResponse.nextChallenge = 
        master.nextResponse.nextChallenge = challenge
    } else if (m.slice(0, 'connectResponse'.length).toString('utf-8').toLowerCase() == 'connectresponse') {
        var challenge = mergeMaster(Object.assign({
            // begin netchan compression
            connected: true,
            channel: {
                compat: false,
                incomingSequence: 0,
                fragmentSequence: 0,
                serverSequence: 0,
                outgoingSequence: 0,
                reliableSequence: 0,
                reliableCommands: [],
                challenge: m.slice('connectResponse'.length).toString('utf-8').trim().split(/\s+/ig)[0]
            }
        }, {
            ip: rinfo.address,
            port: rinfo.port
        }))
        nextResponse.nextConnect = 
        master.nextResponse.nextConnect = challenge
    } else {
        console.log('unknown message:', m.toString('utf-8'))
    }
    nextResponse.nextAny = 
    master.nextResponse.nextAny = {
        content: m.toString('utf-8')
    }
    Object.assign(nextResponse.nextAny, {
        ip: rinfo.address,
        port: rinfo.port
    })
    if(!nextResponse.nextAll) {
        nextResponse.nextAll = []
    }
    if(!master.nextResponse.nextAll) {
        master.nextResponse.nextAll = []
    }
    nextResponse.nextAll.push(nextResponse.nextAny)
    master.nextResponse.nextAll.push(nextResponse.nextAny)
}

async function getNextResponse(key, address, port = 27960) {
    var timeout = 0
    var server
    if(address) {
        var dstIP = await lookupDNS(address)
        server = mergeMaster({
            ip: dstIP,
            port: port
        })
        if(!server.nextResponse) {
            server.nextResponse = {}
        }
        server.nextResponse.nextAll = []
        server.nextResponse[key] = null
    }
    nextResponse.nextAll = []
    nextResponse[key] = null
    return new Promise(resolve => {
        var waiter
        waiter = setInterval(() => {
            timeout += 20
            if((!address && nextResponse[key] != null)
               || (address && server.nextResponse[key] != null)
               || timeout >= MAX_TIMEOUT) {
                clearInterval(waiter)
                resolve(nextResponse[key])
            }
        }, 20)
    })
}

async function nextInfoResponse(address, port = 27960) {
    return await getNextResponse('nextInfo', address, port)
}

async function nextServerResponse(address, port = 27960) {
    return await getNextResponse('nextServer', address, port)
}

async function nextStatusResponse(address, port = 27960) {
    return await getNextResponse('nextStatus', address, port)
}

async function nextPrintResponse(address, port = 27960) {
    return await getNextResponse('nextPrint', address, port)
}

async function nextChallengeResponse(address, port = 27960) {
    return await getNextResponse('nextChallenge', address, port)
}

async function nextChannelMessage(address, port = 27960) {
    return await getNextResponse('nextChannel', address, port)
}

async function nextGamestate(address, port = 27960) {
    return await getNextResponse('nextGamestate', address, port)
}

async function nextConnectResponse(address, port = 27960) {
    return await getNextResponse('nextConnect', address, port)
}

async function nextSnapshot(address, port = 27960) {
    return await getNextResponse('nextSnapshot', address, port)
}

async function nextChat(address, port = 27960) {
    return await getNextResponse('nextChat', address, port)
}

async function nextAnyResponse(address, port = 27960) {
    return await getNextResponse('nextAny', address, port)
}

async function nextAllResponses(address, port = 27960) {
    return await getNextResponse('nextAll', address, port)
}

async function getChallenge(address, port = 27960, challenge, gamename) {
    var dstIP = await lookupDNS(address)
    var msgBuff = new Buffer.from(`\xFF\xFF\xFF\xFFgetchallenge ${challenge} ${gamename}`.split('').map(c => c.charCodeAt(0)))
    udpClient.send(msgBuff, 0, msgBuff.length, port, dstIP)
}

async function sendConnect(address, port = 27960, info) {
    var connectInfo = typeof info == 'string' 
        ? info 
        : Object.keys(info).map(k => '\\' + k + '\\' + info[k]).join('')
    var dstIP = await lookupDNS(address)
    var compressedInfo = await compressMessage(`\xFF\xFF\xFF\xFFconnect "${connectInfo}"`)
    var msgBuff = new Buffer.from(compressedInfo)
    udpClient.send(msgBuff, 0, msgBuff.length, port, dstIP)
}


function NETCHAN_GENCHECKSUM(challenge, sequence) {
    return (challenge) ^ ((sequence) * (challenge))
}

var pak8pk3 = [0,695294960,269430381,2656948387,485997170,1095318617]

async function sendSequence(address, port, channel) {
    var msg
    //console.log('seq', channel.serverSequence, channel.commandSequence)
    msg = writeBits([]    , 0     , channel.serverId || 0, 32)
    msg = writeBits(msg[1], msg[0], channel.serverId ? (channel.serverSequence || 0) : 0, 32)
    msg = writeBits(msg[1], msg[0], channel.serverId ? (channel.commandSequence || 0) : 0, 32)
    
    // write any unacknowledged clientCommands
    for ( var i = channel.reliableAcknowledge + 1 ; i <= channel.reliableSequence ; i++ ) {
		msg = writeBits(msg[1], msg[0], 4, 8) // clc_clientCommand
		msg = writeBits(msg[1], msg[0], i, 32)
        var command = channel.reliableCommands[ i & (MAX_RELIABLE_COMMANDS-1) ]
        for ( var c = 0 ; c < command.length; c++ ) {
            // get rid of 0x80+ and '%' chars, because old clients don't like them
            var v
            if ( command[c] & 0x80 || command[c] == '%' )
                v = '.'.charCodeAt(0);
            else
                v = command[c].charCodeAt(0);
            msg = writeBits(msg[1], msg[0], v, 8)
        }
        msg = writeBits(msg[1], msg[0], 0, 8)
	}

    // empty movement
    msg = writeBits(msg[1], msg[0], 3, 8) // clc_moveNoDelta
    // write the command count
    msg = writeBits(msg[1], msg[0], 1, 8)
    // write delta user cmd key
    msg = writeBits(msg[1], msg[0], 1, 1)
    // TODO: spectate and record player locations
    msg = writeBits(msg[1], msg[0], 0, 1) // no change

    var dstIP = await lookupDNS(address)
    var qport = udpClient.address().port
    var checksum = NETCHAN_GENCHECKSUM(channel.challenge, channel.outgoingSequence)
    var msgBuff = Buffer.concat([new Buffer.from([
        (channel.outgoingSequence >> 0) & 0xFF,
        (channel.outgoingSequence >> 8) & 0xFF,
        (channel.outgoingSequence >> 16) & 0xFF,
        (channel.outgoingSequence >> 24) & 0xFF,
        (qport >> 0) & 0xFF,
        (qport >> 8) & 0xFF,
        (checksum >> 0) & 0xFF,
        (checksum >> 8) & 0xFF,
        (checksum >> 16) & 0xFF,
        (checksum >> 24) & 0xFF,
    ]), msg[1]])
    channel.outgoingSequence++
    //console.log(qport, channel.commandSequence)
    udpClient.send(msgBuff, 0, msgBuff.length, port, dstIP)
}

async function sendReliable(address, port, cmd) {
    var dstIP = await lookupDNS(address)
    var server = mergeMaster({
        ip: dstIP,
        port: port
    })
    if(typeof server.channel != 'undefined') {
        console.log('clientCommand: ', cmd)
        var channel = server.channel
    	channel.reliableSequence++
        var index = channel.reliableSequence & ( MAX_RELIABLE_COMMANDS - 1 )
        channel.reliableCommands[index] = cmd
        await sendSequence(address, port, channel)
    } else
        console.log('Not connected')
}

async function sendPureChecksums(address, port, channel) {
    // TODO: calculate different checksums for other games QVMs
    var checksum = pak8pk3[0] = channel.checksumFeed
    var headers = new Uint8Array(Uint32Array.from(pak8pk3).buffer)
    var digest = new Uint32Array(4)
    mdfour(digest, headers, headers.length)
    var unsigned = new Uint32Array(1)
    unsigned[0] = digest[0] ^ digest[1] ^ digest[2] ^ digest[3]
    checksum ^= unsigned[0]
    checksum ^= 1
    sendReliable(address, port, 'cp ' + channel.serverId 
                 + ' '   + unsigned[0]
                 + ' '   + unsigned[0]
                 + ' @ ' + unsigned[0]
                 + ' '   + checksum)
}

async function sendRcon(address, port = 27960, command, password = DEFAULT_PASS) {
    var dstIP = await lookupDNS(address)
    var msgBuff = new Buffer.from(`\xFF\xFF\xFF\xFFrcon "${password}" ${command}`.split('').map(c => c.charCodeAt(0)))
    udpClient.send(msgBuff, 0, msgBuff.length, port, dstIP)
}

async function getStatus(address, port = 27960) {
    var dstIP = await lookupDNS(address)
    var msgBuff = new Buffer.from('\xFF\xFF\xFF\xFFgetstatus'.split('').map(c => c.charCodeAt(0)))
    udpClient.send(msgBuff, 0, msgBuff.length, port, dstIP)
}

async function getInfo(address, port = 27960) {
    var dstIP = await lookupDNS(address)
    var msgBuff = new Buffer.from('\xFF\xFF\xFF\xFFgetinfo xxx'.split('').map(c => c.charCodeAt(0)))
    udpClient.send(msgBuff, 0, msgBuff.length, port, dstIP)
}

async function listMasters(master = DEFAULT_MASTER, port = 27950, wait = true) {
    var dstIP = await lookupDNS(master)
    var msgBuff = new Buffer.from('\xFF\xFF\xFF\xFFgetservers 68 empty'.split('').map(c => c.charCodeAt(0)))
    udpClient.send(msgBuff, 0, msgBuff.length, port, dstIP)
    if(wait) {
        await new Promise(resolve => setTimeout(resolve, MAX_TIMEOUT))
    } else {
        var timeout = 0
        var timer
        // can't use nextInfoResponse() because it depends on at least 1 statusResponse
        await nextStatusResponse()
    }
    return masters
}

module.exports = {
    listMasters,
    getInfo,
    getStatus,
    getChallenge,
    sendRcon,
    sendConnect,
    sendSequence,
    sendReliable,
    sendPureChecksums,
    udpClient,
    nextInfoResponse,
    nextStatusResponse,
    nextServerResponse,
    nextPrintResponse,
    nextChallengeResponse,
    nextConnectResponse,
    nextChannelMessage,
    nextGamestate,
    nextAnyResponse,
    nextSnapshot,
    nextChat,
    nextAllResponses,
}


#### test quake 3 rcon commands?



In [ ]:
var importer = require('../Core')
var serverApi = importer.import('quake 3 server commands')

async function testRcon (command) {
    var masters = await serverApi.listMasters(void 0, void 0, false)
    console.log(masters)
    await serverApi.sendRcon(masters[0].ip, masters[0].port, command)
    await new Promise(resolve => setTimeout(resolve, 1000))
    serverApi.udpClient.close()
}

module.exports = testRcon


### quake 3 server responses



#### the code

quake 3 server responses?



In [ ]:

function getServersResponse(m) {
    var masters = []
    m = m.slice('getserversResponse'.length)
    for(var i = 0; i < m.length / 7; i++) {
        var ip = i*7+1
        if(m[ip-1] !== '\\'.charCodeAt(0)) continue
        if(m.slice(ip, ip+3) == 'EOT') continue
        var master = {
            ip: m[ip] + '.' + m[ip+1] + '.' + m[ip+2] + '.' + m[ip+3],
            port: (m[ip+4] << 8) + m[ip+5],
        }
        masters.push(master)
    }
    return masters
}

function parseConfigStr(m) {
    return m.toString('utf-8')
        .trim().split(/\n/ig)[0]
        .trim().split(/\\/ig).slice(1)
        .reduce((obj, c, i, arr) => {
            if(i & 1) {
                obj[arr[i-1].toLowerCase()] = c
                obj[arr[i-1]] = c
            }
            return obj
        }, {})
}

function statusResponse(m) {
    m = m.slice('statusResponse'.length)
    var status = parseConfigStr(m)
    var players = m.toString('utf-8')
        .trim().split(/\n/ig)
        .slice(1)
        .reduce((obj, c, i, arr) => {
            if(c.trim().length == 0)
                return obj
            var player = {
                'i': (i + 1),
                'name': (/[0-9]+\s+[0-9]+\s+"(.*)"/ig).exec(c)[1],
                'score': (/([0-9]+)\s+/ig).exec(c)[1],
                'ping': (/[0-9]+\s+([0-9]+)\s+/ig).exec(c)[1],
            }
            player.isBot = player.ping == 0
            obj.push(player)
            return obj
        }, [])
    return Object.assign(status, {players})
}

function infoResponse(m) {
    m = m.slice('infoResponse'.length)
    return parseConfigStr(m)
}

module.exports = {
    getServersResponse,
    statusResponse,
    infoResponse,
    parseConfigStr,
}


### format quake 3 response?



#### the code



In [ ]:
var removeCtrlChars = importer.import('remove ctrl characters')
var importer = require('../Core')

function formatQuake3Response(response, command, server) {
    // try to detect format from response
    var map = (/map:\s(.+)$/igm).exec(response)
    var status = response.match(/name/ig) && response.match(/ping/ig)
    var div = (/^[\-\s]+$/igm).exec(response)
    var players = importer.regexToArray(/^\s*([0-9]+)\s+([0-9]+)\s+([0-9]+)\s+([^\s]+)\s+([^\s]+)\s+.*?$/igm, response, false)
    if(map && status && div) {
        server.mapname = map[1]
        return {
            embeds: [{
                title: removeCtrlChars(server.sv_hostname || server.hostname || server.gamename || server.game || ''),
                description: server.ip + ':' + server.port,
                color: 0xdda60f,
                fields: [
                    {
                        name: 'Map',
                        value: map[1],
                        inline: false
                    },
                    {
                        name: 'Players',
                        value: server.clients + '/' + server.sv_maxclients,
                        inline: false
                    },
                    {
                        name: 'Player',
                        value: '```http\n' + players.map((p, i) => i + ') ' + removeCtrlChars(p[4])).join('\n') + '```',
                        inline: true
                    },
                    {
                        name: 'Score',
                        value: '```yaml\n' + players.map(p => p[2]).join('\n') + '```',
                        inline: true
                    },
                    {
                        name: 'Ping',
                        value: '```fix\n' + players.map(p => p[3]).join('\n') + '```',
                        inline: true
                    }
                ]
            }]
        }
    }
    return '\n```\n' + response + '\n```\n'
}

module.exports = formatQuake3Response


## utilities



### decode client messages?


#### the code


In [ ]:
var {readBits} = importer.import('huffman decode')
var {parseConfigStr} = importer.import('quake 3 server responses')

var BIG_INFO_STRING = 8192
var MAX_STRING_CHARS = 1024
var MAX_PACKETLEN = 1400
var FRAGMENT_SIZE = (MAX_PACKETLEN - 100)
var MAX_MSGLEN = 16384
var FLOAT_INT_BITS = 13
var FLOAT_INT_BIAS = (1<<(FLOAT_INT_BITS-1))
var	CS_SERVERINFO = 0 // an info string with all the serverinfo cvars
var CS_SYSTEMINFO = 1 // an info string for server system to client system configuration (timescale, etc)
var MAX_RELIABLE_COMMANDS = 64
var GENTITYNUM_BITS = 10
var MAX_POWERUPS = 16


function SwapLong(read, message) {
    return (message[(read>>3)+3] << 24) + (message[(read>>3)+2] << 16)
        + (message[(read>>3)+1] << 8) + message[(read>>3)]
}

function SwapShort(read, message) {
    return (message[(read>>3)+1] << 8) + message[(read>>3)]
}

function ReadString(read, message, big = false) {
    var result = ''
    do {
        read = readBits( message, read[0], 8 ) // use ReadByte so -1 is out of bounds
        var c = read[1]
        if ( c <= 0 /*c == -1 || c == 0 */
            || (!big && result.length >= MAX_STRING_CHARS-1 )
            || (big && result.length >= BIG_INFO_STRING-1 ) ) {
            break
        }
        // translate all fmt spec to avoid crash bugs
        if ( c == '%' ) {
            c = '.'
        } else
        // don't allow higher ascii values
        if ( c > 127 ) {
            c = '.'
        }
        result += String.fromCharCode(c)
    } while ( true )
    return [read[0], result]
}

function NETCHAN_GENCHECKSUM(challenge, sequence) {
    return (challenge) ^ ((sequence) * (challenge))
}

var entityStateFields = [
    32, //  NETF(pos.trTime)
    0, //  NETF(pos.trBase[0])
    0, //  NETF(pos.trBase[1])
    0, //  NETF(pos.trDelta[0])
    0, //  NETF(pos.trDelta[1])
    0, //  NETF(pos.trBase[2])
    0, //  NETF(apos.trBase[1])
    0, //  NETF(pos.trDelta[2])
    0, //  NETF(apos.trBase[0])
    10, //  NETF(event)
    0, //  NETF(angles2[1])
    8, //  NETF(eType)
    8, //  NETF(torsoAnim)
    8, //  NETF(eventParm)
    8, //  NETF(legsAnim)
    GENTITYNUM_BITS, //  NETF(groundEntityNum)
    8, //  NETF(pos.trType)
    19, //  NETF(eFlags)
    GENTITYNUM_BITS, //  NETF(otherEntityNum)
    8, //  NETF(weapon)
    8, //  NETF(clientNum)
    0, //  NETF(angles[1])
    32, //  NETF(pos.trDuration)
    8, //  NETF(apos.trType)
    0, //  NETF(origin[0])
    0, //  NETF(origin[1])
    0, //  NETF(origin[2])
    24, //  NETF(solid)
    MAX_POWERUPS, //  NETF(powerups)
    8, //  NETF(modelindex)
    GENTITYNUM_BITS, //  NETF(otherEntityNum2)
    8, //  NETF(loopSound)
    8, //  NETF(generic1)
    0, //  NETF(origin2[2])
    0, //  NETF(origin2[0])
    0, //  NETF(origin2[1])
    8, //  NETF(modelindex2)
    0, //  NETF(angles[0])
    32, //  NETF(time)
    32, //  NETF(apos.trTime)
    32, //  NETF(apos.trDuration)
    0, //  NETF(apos.trBase[2])
    0, //  NETF(apos.trDelta[0])
    0, //  NETF(apos.trDelta[1])
    0, //  NETF(apos.trDelta[2])
    32, //  NETF(time2)
    0, //  NETF(angles[2])
    0, //  NETF(angles2[0])
    0, //  NETF(angles2[2])
    32, //  NETF(constantLight)
    16, //  NETF(frame)
]

async function decodeClientMessage(message, channel) {
    var read = 0
    var sequence = SwapLong(read, message)
    read += 32
    var fragment = (sequence >>> 31) === 1
    if(fragment) {
        sequence &= ~(1 << 31)
    }
    if(false) { // from client to server
        /*qport=*/ read += 16
    }

    var valid = false
    if(!channel.compat) {
      var checksum = SwapLong(read, message)
      read += 32
      valid = NETCHAN_GENCHECKSUM(channel.challenge, sequence) === checksum
    }
    if(!valid) {
        console.log('Invalid message received', sequence, channel.challenge)
        return false
    }
		
    var fragmentStart = 0
    var fragmentLength = 0
    if(fragment) {
      fragmentStart = SwapShort(read, message)
      read += 16
      fragmentLength = SwapShort(read, message)
      read += 16
    }
		
    if ( sequence <= channel.serverSequence ) {
        console.log('Out of order packet', sequence, channel.incomingSequence)
        return false
    }
		
    //channel.dropped = sequence - (channel.incomingSequence+1)
		
    if(fragment) {
        console.log('fragment message')

        // TODO: implement fragment and only return on final message
        if(!channel.fragmentBuffer) channel.fragmentBuffer = Buffer.from([])
        if(sequence != channel.fragmentSequence) {
            channel.fragmentSequence = sequence
            channel.fragmentLength = 0
            channel.fragmentBuffer = Buffer.from([])
        }

        if ( fragmentStart != channel.fragmentLength ) {
            return false
        }

        channel.fragmentBuffer = Buffer.concat([
            Buffer.from(channel.fragmentBuffer),
            Buffer.from(message.subarray(read>>3, (read>>3) + fragmentLength))
        ])
        channel.fragmentLength += fragmentLength

        if ( fragmentLength == FRAGMENT_SIZE ) {
            return false
        }

        if ( channel.fragmentLength > MAX_MSGLEN ) {
            return false
        }

        // make sure the message sequence is still there
        message = Buffer.concat([
            new Uint8Array(4),
            Buffer.from(channel.fragmentBuffer)
        ])
        read = 32
        channel.fragmentBuffer = Buffer.from([])
        channel.fragmentLength = 0
    }

    channel.serverSequence = sequence
    // finished parsing header

    // get the reliable sequence acknowledge number
    read = readBits(message, read, 32)
    channel.reliableAcknowledge = read[1]
	if ( channel.reliableAcknowledge < channel.reliableSequence - MAX_RELIABLE_COMMANDS ) {
		channel.reliableAcknowledge = channel.reliableSequence
	}

    // parse the message
    while(true) {
        read = readBits(message, read[0], 8)
        var cmd = read[1]

        if ( cmd == 8 ) { // svc_EOF
            break;
        }

        channel.messageType = cmd
        switch(cmd) {
            default:
                console.log('Illegible server message', cmd)
            break
            case 0: // svc_bad
            break
            case 1: // svc_nop
            break
            case 2: // svc_gamestate
                read = readBits(message, read[0], 32)
                channel.commandSequence = read[1]
                while(true) {
                    read = readBits(message, read[0], 8)
                    if(read[1] == 8)  // svc_EOF
                        break

                    switch(read[1]) {
                        default: 
                            console.log('Bad command byte')
                        break
                        case 3: // svc_configstring
                            read = readBits(message, read[0], 16)
                            var csI = read[1]
                            read = ReadString(read, message, true /* big */)
                            if(typeof channel.configStrings == 'undefined')
                                channel.configStrings = []
                            channel.configStrings[csI] = read[1]
                        break
                        case 4: // svc_baseline
                            if(typeof channel.entities == 'undefined')
                                channel.entities = []
                            read = readBits(message, read[0], GENTITYNUM_BITS)
                            var number = read[1]

                            // delta entity
                            
                            // check for a remove
                            read = readBits(message, read[0], 1)
                            if(read[1] == 1) { // remove
                                delete channel.entities[number]
                                break
                            }
                            
                            // check for no delta
                            read = readBits(message, read[0], 1)
                            if(read[1] == 0) { // no delta
                                if(typeof channel.entities[number] == 'undefined')
                                    channel.entities[number] = 0
                                break
                            }
                            
                            
                            read = readBits(message, read[0], 8)
                            var lc = read[1]

                            for(var j = 0; j < lc; j++) {
                                read = readBits(message, read[0], 1)
                                if(read[1] == 0) // no change
                                    continue

                                // fields bits
                                if(entityStateFields[j] == 0) {
                                    read = readBits(message, read[0], 1)
                                    if(read[1] == 0) {
                                        channel.entities[number] = 0
                                    } else {
                                        read = readBits(message, read[0], 1)
                                        if(read[1] == 0) {
                                            read = readBits(message, read[0], FLOAT_INT_BITS)
                                            channel.entities[number] = read[1] - FLOAT_INT_BIAS
                                        } else {
                                            read = readBits(message, read[0], 32)
                                            channel.entities[number] = read[1]
                                        }
                                    }
                                } else {
                                    read = readBits(message, read[0], 1)
                                    if(read[1] == 0) {
                                        channel.entities[number] = 0
                                    } else {
                                        read = readBits(message, read[0], entityStateFields[j])
                                        channel.entities[number] = read[1]
                                    }
                                }
                            }
                        break
                    }
                }

                read = readBits(message, read[0], 32)
                channel.clientNum = read[1]

                read = readBits(message, read[0], 32)
                channel.checksumFeed = read[1]

                // parse server info
                channel.serverInfo = parseConfigStr(channel.configStrings[CS_SERVERINFO])

                // parse system info
                channel.systemInfo = parseConfigStr(channel.configStrings[CS_SYSTEMINFO])
                channel.serverId = channel.systemInfo['sv_serverid']
                channel.isPure = channel.systemInfo['sv_pure'] == '1'

            break
            case 3: // svc_configstring
            break
            case 4: // svc_baseline
            break
            case 5: // svc_serverCommand
                if(typeof channel.serverCommands == 'undefined')
                    channel.serverCommands = []
                read = readBits(message, read[0], 32)
                var seq = read[1]
                channel.commandSequence = seq
                var index = seq & (MAX_RELIABLE_COMMANDS-1)
                read = ReadString(read, message)
                channel.serverCommands[index] = read[1]
            break
            case 6: // svc_download
            break
            case 7: // svc_snapshot
                // begin sending input messages so we can receive a gamestate
                //   all this does is echo the ACK number back to the server
                // TODO: properly parse snapshot and read XYZ locations
                /*
                
                // read delta playerstate
                read = readBits(message, read[0], 1)
                if(read[1] == 1) {

                    // parse stats
                    read = readBits(message, read[0], 1)
                    if(read[1] == 1) {
                        bits = MSG_ReadBits (msg, MAX_STATS);

                        for (i=0 ; i<MAX_STATS ; i++) {
                            if (bits & (1<<i) ) {
                                to->stats[i] = MSG_ReadShort(msg);
                            }
                        }
                    }
                }

    if ( MSG_ReadBits( msg, 1 ) ) {
        LOG("PS_STATS");
    }

    // parse persistant stats
    if ( MSG_ReadBits( msg, 1 ) ) {
        LOG("PS_PERSISTANT");
        bits = MSG_ReadBits (msg, MAX_PERSISTANT);
        for (i=0 ; i<MAX_PERSISTANT ; i++) {
            if (bits & (1<<i) ) {
                to->persistant[i] = MSG_ReadShort(msg);
            }
        }
    }

    // parse ammo
    if ( MSG_ReadBits( msg, 1 ) ) {
        LOG("PS_AMMO");
        bits = MSG_ReadBits (msg, MAX_WEAPONS);
        for (i=0 ; i<MAX_WEAPONS ; i++) {
            if (bits & (1<<i) ) {
                to->ammo[i] = MSG_ReadShort(msg);
            }
        }
    }

    // parse powerups
    if ( MSG_ReadBits( msg, 1 ) ) {
        LOG("PS_POWERUPS");
        bits = MSG_ReadBits (msg, MAX_POWERUPS);
        for (i=0 ; i<MAX_POWERUPS ; i++) {
            if (bits & (1<<i) ) {
                to->powerups[i] = MSG_ReadLong(msg);
            }
        }
    }
}
*/
                return channel
            break
            case 9: // svc_voipSpeex
            break
            case 10: // svc_voipOpus
            break
            case 16: // svc_multiview
            break
            case 17: // svc_zcmd
            break
        }
    }
    return channel
}

module.exports = decodeClientMessage


### Huffman decoder



#### the code

huffman decode?


In [ ]:
var fs = require('fs')
var huffman = fs.readFileSync('/Users/briancullinan/planet_quake/build/debug-js-js/huffman_js.wasm')
//var Huffman = require('/Users/briancullinan/planet_quake/code/xquakejs/lib/huffman.js')
var Huff_Compress
var Huff_Decompress
var HuffmanGetBit
var HuffmanGetSymbol
var isInit = false

var MAX_MSGLEN = 16384
var buffer
var memory


// negative bit values include signs
function writeBits( msgBytes, offset, value, bits ) {
    var base = 8192 * 12
    var bitIndex = offset
    var nbits = bits&7

	if ( bits < 0 ) {
		bits = -bits
	}
    for(var j = 0; j < MAX_MSGLEN; j++) {
        if(j < msgBytes.length)
            memory[base + j] = msgBytes[j] & 0xFF
        else
            memory[base + j] = 0
    }

    value &= (0xffffffff>>(32-bits))
    if ( nbits ) {
        for ( var i = 0; i < nbits ; i++ ) {
            HuffmanPutBit( base, bitIndex, (value & 1) )
            bitIndex++
            value = (value>>1)
        }
        bits = bits - nbits
    }
    if ( bits ) {
        for( var i = 0 ; i < bits ; i += 8 ) {
            bitIndex += HuffmanPutSymbol( base, bitIndex, (value & 0xFF) )
            value = (value>>8)
        }
    }
    return [bitIndex, memory.slice(base, base + (bitIndex>>3)+1)]
}


function readBits(m, offset, bits = 8) {
    var base = 8192 * 12
    var value
    var nbits = bits & 7
    var sym = base - 4
    var bitIndex = offset
    for(var i = 0; i < m.length; i++)
        memory[base + i] = m[i]
    if ( nbits )
    {
        for ( i = 0; i < nbits; i++ ) {
            value |= HuffmanGetBit( base, bitIndex ) << i
            bitIndex++
        }
        bits -= nbits
    }
    if ( bits )
    {
        for ( i = 0; i < bits; i += 8 )
        {
            bitIndex += HuffmanGetSymbol( sym, base, bitIndex )
            value |= ( memory[sym] << (i+nbits) )
        }
    }
    return [bitIndex, value]
}

async function decompressMessage(message, offset) {
    if(!isInit)
        await init()
    if(typeof message == 'string')
        message = message.split('')
    for(var i = 0; i < message.length; i++)
        Huffman.HEAP8[msgData+i] = c
	Huffman.HEAP32[(msg>>2)+5] = message.length
	Huffman._Huff_Decompress( msg, 12 )
	return Huffman.HEAP8.slice(msgData + offset, msgData + Huffman.HEAP32[(msg>>2)+5])
}

async function compressMessage(message) {
    var msg = 8192 * 12
    var msgStart = (msg + 64)
    if(!isInit)
        await init()
    for(var i = msg; i < msgStart + message.length; i++)
    {
        memory[i] = 0
    }
    memory[msg + 12] = msgStart & 255
    memory[msg + 13] = (msgStart >> 8) & 255
    memory[msg + 14] = (msgStart >> 16) & 255
    memory[msg + 15] = (msgStart >> 24) & 255
    memory[msg + 20] = (message.length + 1) & 255
    memory[msg + 21] = ((message.length + 1) >> 8) & 255
    memory[msg + 22] = 0
    memory[msg + 23] = 0

    if(typeof message == 'string')
        message = message.split('')
    for(var i = 0; i < message.length; i++)
        memory[msgStart + i] = message[i].charCodeAt(0)
    memory[msgStart + message.length] = 0

    Huff_Compress(msg, 12)
    var msgLength = (memory[msg + 21] << 8) + memory[msg + 20]
    var compressed = memory.slice(msgStart, msgStart + msgLength)
    return compressed
}

async function init() {
    var binary = new Uint8Array(huffman)
    let imports = {};
    imports['memory'] = new WebAssembly['Memory']( {'initial': 16, 'maximum': 100} )
    memory = new Uint8Array( imports['memory']['buffer'] )
    let program = await WebAssembly.instantiate(binary, { env: imports })
    Huff_Compress = program.instance.exports.Huff_Compress
    Huff_Decompress = program.instance.exports.Huff_Decompress
    HuffmanGetBit = program.instance.exports.HuffmanGetBit
    HuffmanGetSymbol = program.instance.exports.HuffmanGetSymbol
    HuffmanPutBit = program.instance.exports.HuffmanPutBit
    HuffmanPutSymbol = program.instance.exports.HuffmanPutSymbol
}

init()

module.exports = init
Object.assign(init, {
    readBits,
    writeBits,
    decompressMessage,
    compressMessage
})


### monitor q3 servers?

Idempotent function to manage discord threads based on player activity on Q3 servers

#### the code


In [ ]:
var importer = require('../Core')
var serverApi = importer.import('quake 3 server commands')
var {authorizeGateway} = importer.import('authorize discord')
var discordApi = importer.import('discord api')
var removeCtrlChars = importer.import('remove ctrl characters')
var DEFAULT_USERNAME = 'Orbb'

async function getServerChannel(server) {
    
    // get a list of channels to pair gametype up with
    var channels = await discordApi.guildChannels()
    var channel

    // sort ffa/ctf/freeZe
    if(!channel && (server.server_freezetag == '1'
        || server.gamename.toLowerCase() == 'freon'
        || (typeof server.xp_version != 'undefined'
           && server.g_gametype == '8'))) {
        channel = channels.filter(c => c.name.toLowerCase() == 'freeze-tag')[0]
    }
    if(!channel && server.gamename.toLowerCase() == 'defrag') {
        channel = channels.filter(c => c.name.toLowerCase() == 'defrag')[0]
    }
    if(!channel && server.g_gametype == '4') {
        channel = channels.filter(c => c.name.toLowerCase() == 'capture-the-flag')[0]
    }
    if(!channel && server.g_gametype == '3') {
        channel = channels.filter(c => c.name.toLowerCase() == 'team-deathmatch')[0]
    }
    if(!channel && typeof server.xp_version != 'undefined'
        && server.g_gametype == '7') {
        channel = channels.filter(c => c.name.toLowerCase() == 'clan-arena')[0]
    }
    if(!channel && typeof server.xp_version != 'undefined'
        && server.g_gametype == '1') {
        channel = channels.filter(c => c.name.toLowerCase() == '1on1-duel')[0]
    }
    if(!channel) {
        channel = channels.filter(c => c.name.toLowerCase() == 'general')[0]
    }

    return channel
}

async function updateChannelThread(threadName, channel, json) {
    
    // find old threads to reactivate
    var archived = (await discordApi.archivedThreads(channel.id)).threads 
        .filter(t => t.name == threadName)

    var thread
    var removeOld
    if(archived.length > 0) {
        thread = archived[0]
        removeOld = (await discordApi.getPins(thread.id))
            .filter(p => p.author.username == DEFAULT_USERNAME)
    } else {
        // thread is already active
        var active = (await discordApi.activeThreads(channel.id)).threads
            .filter(t => t.name == threadName)
        if(active.length > 0) {
            // find and update previous "whos online" message, pins?
            thread = active[0]
            var pins = (await discordApi.getPins(thread.id))
                .filter(p => p.author.username == DEFAULT_USERNAME)
            if(pins.length > 0) {
                await discordApi.updateMessage(json, pins[0].id, thread.id)
                return thread
            }
        } else {
            thread = await discordApi.createThread(threadName, channel.id)
        }
    }
    // create new "whos online message"
    var message = await discordApi.createMessage(json, thread.id)
    await discordApi.pinMessage(message.id, thread.id)
    if(removeOld && removeOld.length > 0) {
        await discordApi.unpinMessage(removeOld[0].id, thread.id)
    }
    return thread
}


async function monitorServer(address = 'q3msk.ru', port = 27977) {
    await serverApi.getInfo(address, port)
    await serverApi.getStatus(address, port)
    var server = await serverApi.nextStatusResponse(address, port)
    if(!server || server.monitorRunning)
        return
    server.monitorRunning = true

    var threadName = 'Pickup for ' + removeCtrlChars(server.sv_hostname || server.hostname).replace(/[^0-9a-z-]/ig, '-')
    console.log(threadName)
    var redTeam = (server.Players_Red || '').trim()
        .split(/\s+/ig).filter(n => n)
        .map(i => parseInt(i))
    var blueTeam = (server.Players_Blue || '').trim()
        .split(/\s+/ig).filter(n => n)
        .map(i => parseInt(i))
    server.players.forEach(p => {
        if(redTeam.includes(p.i))
            p.team = 'red'
        else if (blueTeam.includes(p.i))
            p.team = 'blue'
        else
            p.team = 'other'
    })
    server.players.sort((a, b) => {
        return a.team - b.team
    })
    var json
    if(redTeam.length > 0 || blueTeam.length > 0) {
        json = {
            embeds: [{
                title: removeCtrlChars(server.sv_hostname || server.hostname || server.gamename || server.game || ''),
                description: server.ip + ':' + server.port,
                color: 0xdda60f,
                fields: [
                    {
                        name: 'Map',
                        value: server.mapname,
                        inline: false
                    },
                    {
                        name: 'Players',
                        value: server.players.length + '/' + server.sv_maxclients,
                        inline: false
                    },
                    {
                        name: 'Player',
                        value: ':red_circle: Red Team\n```http\n' 
                            + server.players.filter(p => p.team == 'red').map((p, i) => (p.i) + ') ' 
                            + removeCtrlChars(p.name)).join('\n') + '\u0020\n```\n'
                            + ':blue_circle: Blue Team\n```http\n' 
                            + server.players.filter(p => p.team == 'blue').map((p, i) => (p.i) + ') ' 
                            + removeCtrlChars(p.name)).join('\n') + '\u0020\n```\n'
                            + 'Other\n```http\n' 
                            + server.players.filter(p => p.team == 'other').map((p, i) => (p.i) + ') ' 
                            + removeCtrlChars(p.name)).join('\n') + '\u0020\n```\n',
                        inline: true
                    },
                    {
                        name: 'Score',
                        value: '-\n```yaml\n' + server.players.filter(p => p.team == 'red').map(p => p.score).join('\n') + '\u0020\n```'
                            + '-\n```yaml\n' + server.players.filter(p => p.team == 'blue').map(p => p.score).join('\n') + '\u0020\n```'
                            + '-\n```yaml\n' + server.players.filter(p => p.team == 'other').map(p => p.score).join('\n') + '\u0020\n```',
                        inline: true
                    },
                    {
                        name: 'Ping',
                        value: '-\n```fix\n' + server.players.filter(p => p.team == 'red').map(p => p.ping).join('\n') + '\u0020\n```'
                            + '-\n```fix\n' + server.players.filter(p => p.team == 'blue').map(p => p.ping).join('\n') + '\u0020\n```'
                            + '-\n```fix\n' + server.players.filter(p => p.team == 'other').map(p => p.ping).join('\n') + '\u0020\n```',
                        inline: true
                    }
                ]
            }]
        }        
    } else {
        json = {
            embeds: [{
                title: removeCtrlChars(server.sv_hostname || server.hostname || server.gamename || server.game || ''),
                description: server.ip + ':' + server.port,
                color: 0xdda60f,
                fields: [
                    {
                        name: 'Map',
                        value: server.mapname,
                        inline: false
                    },
                    {
                        name: 'Players',
                        value: server.players.length + '/' + server.sv_maxclients,
                        inline: false
                    },
                    {
                        name: 'Player',
                        value: '```http\n' + server.players.map((p, i) => (p.i) + ') ' 
                            + removeCtrlChars(p.name)).join('\n') + '\u0020\n```',
                        inline: true
                    },
                    {
                        name: 'Score',
                        value: '```yaml\n' + server.players.map(p => p.score).join('\n') + '\u0020\n```',
                        inline: true
                    },
                    {
                        name: 'Ping',
                        value: '```fix\n' + server.players.map(p => p.ping).join('\n') + '\u0020\n```',
                        inline: true
                    }
                ]
            }]
        }
    }

    // TODO: comment this line out when launched from index? monitor script
    await authorizeGateway()
    var channel = getServerChannel(server)
    var thread
    if(!channel) {
        console.log('No channel to create thread on.')
    } else {
        thread = await updateChannelThread(threadName, channel, json)
        server.channelId = thread.id
    }
    server.monitorRunning = setTimeout(() => {
        monitorServer(address, port)
    }, 60 * 1000)
    return thread
}

module.exports = monitorServer


### cron entry



#### the code

index?

discord bot index?


In [ ]:
var importer = require('../Core')
var respondCommand = importer.import('respond discord commands')
var monitorServer = importer.import('monitor q3 servers')
var spectateServer = importer.import('spectate q3 server')

var DEFAULT_CHANNEL = process.env.DEFAULT_CHANNEL || 'general'

var serverList = [
    // Defrag
    '83.243.73.220:27961',
    '83.243.73.220:27960',
    '83.243.73.220:27965',
    // Eplus
    '45.32.237.139:27960',
    '45.32.237.139:27000',
    '45.32.237.139:6666',
    '45.32.237.139:6000',
    '173.199.75.8:27963',
    '108.61.122.25:27982',
    '212.187.209.123:27965',
    '79.172.212.116:27970',
    // OSP/baseq3
    '193.33.176.30:27960',
    '85.10.201.6:27960',
    '88.198.221.99:27965',
    '88.198.221.99:27960',
    '88.198.221.98:27962',
    '216.86.155.163:27960',
    '216.86.155.161:27960',
    '216.86.155.173:29676',
    '216.86.155.162:27960',
    '69.30.217.148:27960',
    '69.30.217.148:27960',
    '69.30.217.150:27960',
    '69.30.217.149:27960',
    '212.42.38.88:27960',
    '212.42.38.88:27961',
    '212.42.38.88:27962',
    '212.42.38.88:27963',
    '212.42.38.88:27967',
    '79.142.106.99:27960',
    // CPMA
    '82.196.10.31:27960',
    '45.63.78.66:27970',
    // Msk
    'meat.q3msk.ru:7700',
    'q3msk.ru:27961',
    'q3msk.ru:27962',
    'q3msk.ru:27963',
    'q3msk.ru:27964',
    'q3msk.ru:27965',
    'q3msk.ru:27977',
    'q3msk.ru:27978',
    'tdm.q3msk.ru:27960',
    'ca.q3msk.ru:27960',
    'ca.q3msk.ru:27961',
    'ca.q3msk.ru:27963',
    'pl.q3msk.ru:27962',
    'pl.q3msk.ru:27964',
    'ctf.q3msk.ru:27960',
    'ctf.q3msk.ru:27970',
    'n2.q3msk.ru:29000',
    'q3msk.ru:27980',
    'q3msk.ru:27981',
    'q3msk.ru:27985',
    // QooL7
    'quakearea.com:27960',
    'q3.rofl.it:27960',
]
serverList.forEach(async (s) => {
    var address = s.split(':')[0]
    var port = parseInt(s.split(':')[1] || '27960')
    await monitorServer(address, port)
    //await spectateServer(address, port)
})

var stillRunning = false
var commandResponder
async function startResponder() {
    if(stillRunning) {
        console.log('Still running...')
        return
    }
    stillRunning = true
    try {
        await respondCommand(DEFAULT_CHANNEL)
        await respondCommand('@me')
    } catch (e) {
        console.log(e)
    }
    stillRunning = false
    if(!commandResponder)
        commandResponder = setInterval(startResponder, 5000)
}

module.exports = startResponder


### ssh curl command

Run the curl command on a remote server to download files from discord and maps from ws.q3df.org.



#### the code

ssh remote wget?



In [ ]:
var fs = require('fs')
var path = require('path')
var NodeSSH = require('node-ssh')
var ssh = new NodeSSH()
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
var privateKeyPath
if(fs.existsSync('./id_rsa')) {
    privateKeyPath = path.resolve('./id_rsa')
} else {
    privateKeyPath = path.join(PROFILE_PATH, '.ssh/id_rsa')
}

var DEFAULT_SSH = process.env.DEFAULT_SSH || 'okayfun.com'
var DEFAULT_SSH_USER = process.env.DEFAULT_SSH_USER || 'root'

/*
ssh.connect({
  host: DEFAULT_SSH,
  username: DEFAULT_SSH_USER,
  privateKey: privateKeyPath
})
*/

async function remoteGet(url, output, cwd) {
    var options = {
        cwd: cwd,
        onStdout(chunk) {
          console.log('stdoutChunk', chunk.toString('utf8'))
        },
        onStderr(chunk) {
          console.log('stderrChunk', chunk.toString('utf8'))
        },
    }
    try {
        await ssh.exec('/usr/bin/wget', ['-O', output, url], options)
        await ssh.exec(`
fileLength=$(wc -l ../index.json | cut -d' ' -f1);
sed "$((fileLength-1))s/$/,/;
${fileLength}i  \\\t\"\":\"\"" ../index.json`, [], options)
        
    } catch (e) {
        console.log(e)
    }
}

module.exports = remoteGet


### dns lookup?



#### the code



In [ ]:
var dns = require('dns')
var _dnsLookup = {}

async function lookupDNS(address) {
  if(typeof _dnsLookup[address] != 'undefined')
    return _dnsLookup[address]
  return new Promise((resolve, reject) => dns.lookup(address, function(err, dstIP) {
    if(err) {
      return reject(err)
    }
    _dnsLookup[address] = dstIP
    return resolve(dstIP)
  }))
}

module.exports = lookupDNS


### remove ctrl characters?



#### the code



In [ ]:

// TODO: use this for server names and player names when matching to discord
function removeCtrlChars(str) {
    return str
        .replace(/\^\^[a-z0-9][a-z0-9]/ig, '')
        .replace(/\^[a-z0-9]/ig, '')
        .trim()
}

module.exports = removeCtrlChars


### get server status with gamedig

quake3 server status?



#### the code



In [ ]:
var importer = require('../Core')
var gamedig = require('gamedig')
var serverApi = importer.import('quake 3 server commands')
var { sendRcon, nextAllResponses, udpClient } = importer.import('quake 3 server commands')
var discordApi = importer.import('discord api')
var {authorizeGateway} = importer.import('authorize discord')
var {parseConfigStr} = importer.import('quake 3 server responses')
var removeCtrlChars = importer.import('remove ctrl characters')

async function getStatus(ip, port) {
    return gamedig.query({
        type: 'quake3',
        host: ip,
        port: port
    }).then((state) => {
        return state
    }).catch((error) => {
        console.log('Server is offline', error)
    })
}

async function captureAllStats() {
    var masters = await serverApi.listMasters('master.ioquake3.org', void 0, false)
    //var status = await getStatus(masters[1].ip, masters[1].port)
    var status = await getStatus('45.32.237.139', 27960)
    console.log(status.bots)
}

//typedef enum {
var SV_EVENT = {
	MAPCHANGE: 0,
    CLIENTSAY: 1,
    MATCHEND: 2,
    CALLADMIN: 3,
    CLIENTDIED: 4,
    CLIENTWEAPON: 5,
    CLIENTRESPAWN: 6,
    CLIENTAWARD: 7,
    GETSTATUS: 8,
    SERVERINFO: 9,
    CONNECTED: 10,
    DISCONNECT: 11,
}
//} recentEvent_t;


async function getChats(channelId) {
    var match = (/^(.*?):*([0-9]+)*$/ig).exec()
    await sendRcon('127.0.0.1', 27960, '', 'recentPassword')
    var response = await nextAllResponses()

    if(!response) return

    var maxTime = 0
    var parsed = response.map(function (r) {
        return JSON.parse(r.content)
    })
    var chats = parsed.filter(function (r) {
        if(r.timestamp > maxTime)
            maxTime = r.timestamp
        return r.type == SV_EVENT.CLIENTSAY
    })
    
    var call = parsed.filter(function (r) {
        return r.type == SV_EVENT.CALLADMIN
    })
    
    var status = parsed.filter(function (r) {
        return r.type == SV_EVENT.GETSTATUS
    })
    var server = {}
    if(status.length) {
        Object.assign(server, parseConfigStr(status[0].value))
    }

    var info = parsed.filter(function (r) {
        return r.type == SV_EVENT.SERVERINFO
    })
    if(info.length) {
        Object.assign(server, parseConfigStr(info[0].value))
    }
    
    var match = parsed.filter(function (r) {
        return r.type == SV_EVENT.MATCHEND
    })
    if(match.length) {
        // TODO: save to SQL database
        console.log(match[match.length-1])
    }

    var discordSocket = await authorizeGateway()
    //console.log(await discordApi.getGuildRoles('752561748611039362'))
    if(call.length) {
        await discordApi.triggerTyping(channelId)        
    }
    for(var i = 0; i < call.length; i++) {
        try {
            //console.log('Say: ' + call[i].value)
            await discordApi.createMessage({
                embed: {
                    title: removeCtrlChars(server.hostname || server.sv_hostname || server.gamename),
                    description: server.ip + ':' + server.port,
                    color: 0xdda60f,
                    fields: [
                        {
                            name: call[i].value,
                            value: `<@&752605581029802155> [Connect](https://quake.games/?connect%20${'127.0.0.1:27960'})`,
                            inline: false
                        },
                    ]
                },
                allowed_mentions: {
                    parse: ['users', 'roles'],
                    users: [],
                    roles: []
                }
            }, channelId)
            //await discordApi.createMessage(`@admin ${call[i].value}`, channelId)
        } catch (e) {
            console.log(e)
        }
    }
}

module.exports = getChats

### spectate q3 server?

Simulate a full player connection to spectate a quake 3 match. Stays connected, switches maps, monitors for chat messages.


#### the code


In [ ]:
var importer = require('../Core')
var {
    getInfo, nextInfoResponse,
    getChallenge, nextChallengeResponse,
    udpClient, sendConnect, nextConnectResponse,
    nextChannelMessage, nextGamestate,
    sendPureChecksums, nextSnapshot,
    sendReliable, nextChat
    
} = importer.import('quake 3 server commands')
var discordApi = importer.import('discord api')
var {authorizeGateway} = importer.import('authorize discord')
var removeCtrlChars = importer.import('remove ctrl characters')


async function spectateServer(address = 'localhost', port = 27960) {

    // TODO: comment this line out when launched from index? monitor script
    await authorizeGateway()

    
    var challenge = new ArrayBuffer(4)
    for(var c = 0; c < 4; c++) {
        challenge[c] = Math.round(Math.random() * 255)
    }
    await getInfo(address, port)
    var info = await nextInfoResponse(address, port)
    if(!info)
        return
    await getChallenge(address, port, new Uint32Array(challenge)[0], info.gamename || info.game)
    var challenge = (await nextChallengeResponse(address, port)).challenge
    await sendConnect(address, port, {
        qport: udpClient.address().port,
        challenge: challenge,
        name: 'Orbb-Bot',
        protocol: 71,
    })
    challenge = await nextConnectResponse(address, port)
    var gamestate = await nextGamestate(address, port)
    console.log('gamestate', server.sv_hostname || server.hostname)
    if(!gamestate.channel)
        return
    if(gamestate.isPure) {
        // TODO: send valid "cp" checksums to pure servers
        await sendPureChecksums(address, port, gamestate)
    }
    await nextSnapshot(address, port)
    await sendReliable(address, port, 'team s')

    // await print commands
    info.chatListener = setInterval(async () => {
        if(!info.chatWaiting) {
            info.chatWaiting = true
            var message = await nextChat(address, port)
            info.chatWaiting = false
            // forward print commands to discord
            if(message) {
                message = removeCtrlChars((/"([^"]*?)"/).exec(message)[1])
                discordApi.createMessage(message, info.channelId)
            }
        }
    }, 100)
}


module.exports = spectateServer
